In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

### Loading article df

In [1]:
!pwd

/Users/ravi.tej/Desktop/ML/Recommendations/arcane/notebooks/llms/refined-prompt-30-nov-2023


In [4]:
data_folder = "../../../data/"
import json
with open(data_folder + 'insight_db_articles.json') as f:
    insight_articles = json.load(f)

In [5]:
valid_articles = {}

for k in insight_articles:
    if 'is_premium_article' in k and k['is_premium_article'] == False:
        article_id = k['_id']['$oid']
        valid_articles[article_id] = k
        valid_articles[article_id]['source_id'] = k['source_id']['$oid']

In [6]:
len(valid_articles)

162012

In [7]:
import numpy as np
import pandas as pd

In [8]:
all_articles_df = pd.DataFrame(valid_articles).T.reset_index()
all_articles_df = all_articles_df.rename(columns = {'index': 'article_id'})
all_articles_df = all_articles_df.drop(columns = {'_id', '_class'})

In [9]:
all_articles_df[all_articles_df.groupby('url')['url'].transform('count') > 1].sort_values('url').to_csv('duplicate_articles_df.csv')

In [10]:
articles_df = pd.DataFrame(valid_articles).T.reset_index()
articles_df = articles_df.rename(columns = {'index': 'article_id'})
articles_df = articles_df.drop(columns = {'_id', '_class'})
articles_df['prob'] = [np.random.rand() for i in range(len(articles_df))]

In [11]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [12]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT * FROM source_id_characteristics'
    cursor.execute(sql)
    results = cursor.fetchall()

In [13]:
results

[(4, '652d53256a2736f06f46cfcf', Decimal('1'), 'wint-wealth'),
 (3, '65291b1e9a2fbc229e5f29c9', Decimal('1'), 'zerodha-varsity'),
 (5, '6530d40d9a7559d3ec6b9871', Decimal('1.0'), 'groww'),
 (1, '6512cdcad01a9c8e86263e05', Decimal('0.9'), 'economic-times'),
 (2, '650046bd005149c49201269f', Decimal('0.9'), 'live-mint'),
 (10, '65558d1f672118037c541088', Decimal('1.0'), 'navi'),
 (11, '655597ea96fb32eef21a562a', Decimal('0.9'), 'investopedia'),
 (12, '6555987996fb32eef21a562b', Decimal('0.9'), 'business-standard'),
 (13, '655598ab96fb32eef21a562c', Decimal('0.9'), 'business-insider'),
 (14, '6555992096fb32eef21a562d', Decimal('0.9'), 'cnbc-tv-18'),
 (15, '6555995596fb32eef21a562e', Decimal('0.9'), 'cnbc'),
 (16, '65559d0c96fb32eef21a562f', Decimal('0.9'), 'business-world'),
 (17, '65559d4296fb32eef21a5630', Decimal('0.9'), 'bq-prime'),
 (18, '6560a5bddf69601d3e2ce0c1', Decimal('0.9'), 'bbc'),
 (19, '6560ac8b9952a16bbc397223', Decimal('0.9'), 'yahoo-finance'),
 (20, '6560af039952a16bbc3972

In [14]:
source_df = pd.DataFrame(results, columns = ['id', 'source_id', 'decay', 'source_name'])

In [15]:
articles_df = pd.merge(articles_df, source_df, how = 'left', on = 'source_id')

### Finding any removing duplicate articles

In [16]:
round(100 * (articles_df.groupby('source_name')['url'].count() - articles_df.groupby('source_name')['url'].nunique())/articles_df.groupby('source_name')['url'].count())

source_name
bq-prime             0.0
business-insider    43.0
business-world       5.0
cnbc                49.0
cnbc-tv-18          38.0
economic-times       0.0
groww                0.0
investopedia        50.0
live-mint           42.0
navi                 0.0
wint-wealth          0.0
zerodha-varsity      2.0
Name: url, dtype: float64

In [17]:
articles_df.groupby('url')['url'].count().nlargest(30)

url
https://www.livemint.com/auto-news/the-100cc-bike-rides-into-a-downturn-11669146313134.html                                                                                             10
https://www.livemint.com/auto-news/upcoming-cars-expected-to-debut-this-diwali-season-tata-punch-ev-renault-duster-more-11698740598121.html                                             10
https://www.livemint.com/companies/company-results/gail-india-q2-results-net-profit-jumps-70-to-rs-2-405-crore-revenue-dips-1-3-qoq-11698742923532.html                                 10
https://www.livemint.com/companies/company-results/lt-q2-results-live-updates-l-t-results-2023-l-t-q2-earnings-l-t-q2-profit-l-t-share-price-11698721205148.html                        10
https://www.livemint.com/companies/company-results/lt-q2-results-preview-net-profit-likely-to-rise-25-revenue-to-grow-18-yoy-on-strong-orders-11698723519988.html                       10
https://www.livemint.com/companies/company-results/q2-results

In [18]:
articles_df = articles_df.drop_duplicates('url')

In [19]:
articles_df.groupby('source_name')['url'].count()

source_name
bq-prime              522
business-insider     1646
business-world       6767
cnbc                13778
cnbc-tv-18          11032
economic-times      23795
groww                1731
investopedia        18343
live-mint           23124
navi                 1714
wint-wealth          1743
zerodha-varsity       859
Name: url, dtype: int64

In [20]:
articles_df.groupby('url')['source_name'].nunique().nlargest()

url
https://economictimes.indiatimes.com/industry/auto/auto-components/adoption-of-electric-vehicles-expected-to-accelerate-in-coming-years-rane-brake-linings-ltd/articleshow/101434376.cms             1
https://economictimes.indiatimes.com/industry/auto/auto-components/assurance-intl-goodyear-unveil-new-range-of-filters-batteries-in-india/articleshow/102611689.cms                                  1
https://economictimes.indiatimes.com/industry/auto/auto-components/auto-component-industry-reports-highest-turnover-in-fy23-aims-to-log-double-digit-growth-this-fiscal/articleshow/102501507.cms    1
https://economictimes.indiatimes.com/industry/auto/auto-components/delphi-tvs-technologies-to-take-up-capital-expansion-at-chennai-plant/articleshow/99403761.cms                                    1
https://economictimes.indiatimes.com/industry/auto/auto-components/eveready-industries-cuts-number-of-direct-distributors-to-1000-from-5000-earlier/articleshow/100279158.cms                        1
N

In [88]:
articles_df[articles_df.title == 'A Step-by-Step Guide to UAN Generation and Activation Process'].url.values

array(['https://www.wintwealth.com/blog/a-step-by-step-guide-to-uan-generation-and-activation-process',
       'https://www.wintwealth.com/blog/a-step-by-step-guide-to-uan-generation-and-activation-process-2',
       'https://navi.com/blog/uan/'], dtype=object)

In [89]:
articles_df[articles_df.title == 'Commerce ministry asks exporters to focus on potential sectors, key markets to boost exports'].url.values

array(['https://economictimes.indiatimes.com/news/economy/foreign-trade/commerce-ministry-asks-exporters-to-focus-on-potential-sectors-key-markets-to-boost-exports/articleshow/101464696.cms',
       'https://www.cnbctv18.com/business/commerce-ministry-asks-exporters-to-focus-on-potential-sectors-key-markets-to-boost-exports-17105661.htm'],
      dtype=object)

In [96]:
articles_df[articles_df.title == 'Oil extends gains as OPEC+ to mull deeper cuts'].url.values

array(['https://www.cnbctv18.com/market/oil-extends-gains-as-opec-to-mull-deeper-cuts-18362691.htm',
       'https://www.cnbc.com/2023/11/20/oil-extends-gains-as-opec-to-mull-deeper-cuts.html'],
      dtype=object)

In [94]:
pd.options.display.max_rows = 200

In [95]:
articles_df[articles_df.groupby(['title'])['source_name'].transform('nunique') > 1].groupby('title')['source_name'].unique()

title
A Step-by-Step Guide to UAN Generation and Activation Process                                                     [wint-wealth, navi]
Adani Ports' move to buy back USD 130-million bonds an opportunistic exchange: S&P Global Ratings        [economic-times, cnbc-tv-18]
Adani Wilmar files police complaint against sale of counterfeit Fortune brand products                   [economic-times, cnbc-tv-18]
Airbnb inks pact with tourism ministry to promote heritage stays, cultural tourism                       [economic-times, cnbc-tv-18]
Amtrak train with 198 passengers derails after hitting truck on tracks in Southern California                  [cnbc, economic-times]
B'luru, H'bad, Chennai contribute 59 pc to total office leasing across 7 cities in Apr-Jun: Report       [economic-times, cnbc-tv-18]
BHEL logs over 17% rise in new orders in 2022-23                                                         [economic-times, cnbc-tv-18]
Bajaj Allianz Life declares bonus worth Rs 1,201 crore i

In [54]:
articles_df.groupby('source_name')['url'].count()/len(articles_df)

source_name
bq-prime            0.004969
business-insider    0.015668
business-world      0.064414
cnbc                0.131152
cnbc-tv-18          0.105013
economic-times      0.226503
groww               0.016477
investopedia        0.174605
live-mint           0.220115
navi                0.016315
wint-wealth         0.016591
zerodha-varsity     0.008177
Name: url, dtype: float64

### Extracting cat, subcat

In [57]:
articles_df[articles_df.source_name == 'bq-prime'].sample(5).url.values

array(['https://www.bqprime.com/business/eight-indigo-passengers-left-stranded-in-bengaluru-airline-blames-lack-of-time',
       'https://www.bqprime.com/business/government-cuts-windfall-tax-on-crude-diesel',
       'https://www.bqprime.com/business/maharashtra-needs-153-trillion-investment-in-6-years-for-its-1-trillion-economy-vision',
       'https://www.bqprime.com/business/indigo-parent-interglobe-plans-to-launch-electric-air-taxi-service-in-india-in-2026',
       'https://www.bqprime.com/markets/fpis-stay-net-sellers-for-the-14th-straight-day-3'],
      dtype=object)

In [66]:
# no proper structure cat in business insider
articles_df[articles_df.source_name == 'business-insider'].sample(5).url.values

array(['https://www.businessinsider.com/sc/how-twilio-is-embracing-ai-to-personalize-engagement',
       'https://www.businessinsider.com/guides/learning/princeton-university-free-online-courses',
       'https://www.businessinsider.com/moving-to-leaving-florida-reasons-home-prices-affordability-weather-2023-11',
       'https://www.businessinsider.com/kaiser-permanente-worker-strike-union-patient-health-risk-2023-9',
       'https://www.businessinsider.com/why-apple-has-smarter-pricing-strategy-than-facebook-headset-2023-6'],
      dtype=object)

In [67]:
articles_df[articles_df.source_name == 'business-world'].sample(5).url.values

array(['https://www.bworldonline.com/world/2023/11/16/557974/czech-government-freezes-russian-states-real-estate-assets/',
       'https://www.bworldonline.com/infographics/2023/07/26/535964/philippines-ranks-12th-most-attractive-country-for-offshore-services/',
       'https://www.bworldonline.com/world/2023/07/21/535324/chinese-hackers-breach-us-ambassadors-emails-wsj/',
       'https://www.bworldonline.com/corporate/2023/04/21/518191/globe-smart-still-keen-on-longer-sim-deadline/',
       'https://www.bworldonline.com/agribusiness/2023/10/02/549016/phl-2024-broiler-chicken-production-forecast-at-1-53-million-metric-tons/'],
      dtype=object)

In [71]:
articles_df[articles_df.source_name == 'cnbc'].sample(5).url.values

array(['https://www.cnbc.com/2023/08/31/trump-pleads-not-guilty-to-election-charges-in-georgia-waives-arraignment.html',
       'https://www.cnbc.com/2023/08/24/first-trial-of-trump-co-defendant-in-georgia-case-will-begin-oct-23-judge-rules.html',
       'https://www.cnbc.com/2023/11/02/starbucks-sbux-earnings-q4-2023.html',
       'https://www.cnbc.com/2023/09/17/russell-brands-agency-terminates-ties-following-sexual-assault-allegations.html',
       'https://www.cnbc.com/2023/10/04/health-care-workers-strike-at-kaiser-permanente.html'],
      dtype=object)

In [75]:
articles_df[articles_df.source_name == 'cnbc-tv-18'].sample(5).url.values

array(['https://www.cnbctv18.com/finance/canfin-homes-ceo-exclusive-interview-suresh-srinivasan-iyer-fraud-ambala-branch-cheque-process-17333691.htm',
       'https://www.cnbctv18.com/market/nse-signs-mou-with-maharashtra-moneybee-institute-to-create-financial-literacy-nagpur-devendra-fadnavis-16941571.htm',
       'https://www.cnbctv18.com/market/stocks/tata-teleservices-new-ucaas-launch-microsoft-teams-integration-shares-rally-17275581.htm',
       'https://www.cnbctv18.com/economy/indian-govt-fta-fears-eu-carbon-tariffs-will-raise-compliance-costs-impede-free-trade-17144621.htm',
       'https://www.cnbctv18.com/market/commodities/saxo-bank-forecasts-gold-prices-to-reach-2100-dollar-per-ounce-asset-class-investment-16494651.htm'],
      dtype=object)

In [97]:
articles_df[articles_df.source_name == 'economic-times'].sample(5).url.values

array(['https://economictimes.indiatimes.com/news/india/mumbai-creek-tragedy-bodies-of-2-boys-found-1-still-missing/articleshow/101815438.cms',
       'https://economictimes.indiatimes.com/news/international/us/demon-slayer-season-4-what-you-need-to-know/articleshow/104449422.cms',
       'https://economictimes.indiatimes.com/news/india/heavy-rain-lashes-delhi-waterlogging-causes-traffic-congestion-in-parts-of-city/articleshow/101788655.cms',
       'https://economictimes.indiatimes.com/news/india/sc-trashes-appeal-of-former-isro-scientist-against-dismissal-from-service/articleshow/100232069.cms',
       'https://economictimes.indiatimes.com/news/politics-and-nation/shiv-sena-ubt-leader-sanjay-raut-backs-yogi-adityanath-on-encounters-says-no-question-of-caste-religion-if-up-cm-wants-to-end-mafia-raj/articleshow/99499420.cms'],
      dtype=object)

In [98]:
articles_df[articles_df.source_name == 'groww'].sample(5).url.values

array(['https://groww.in/blog/best-fund-managers-in-india',
       'https://groww.in/blog/apollo-hospitals-q1-results-fy22-2023-pat-down-by-35-to-rs-317-crore',
       'https://groww.in/blog/bandhan-bank-quarterly-results-q3-fy22',
       'https://groww.in/blog/best-dividend-paying-penny-stocks-in-india',
       'https://groww.in/blog/reasons-not-invest-small-cap-funds'],
      dtype=object)

In [107]:
articles_df[articles_df.source_name == 'investopedia'].sample(5).url.values

array(['https://www.investopedia.com/terms/a/at-par.asp',
       'https://www.investopedia.com/articles/financial-advisors/101315/are-etfs-good-fit-401k-plans.asp',
       'https://www.investopedia.com/mortgage-rates-continue-upward-surge-7557401',
       'https://www.investopedia.com/articles/exchangetradedfunds/11/benefits-etfs-hedging.asp',
       'https://www.investopedia.com/articles/personal-finance/120715/what-happens-retirement-accounts-if-spouse-dies.asp'],
      dtype=object)

In [101]:
articles_df[articles_df.source_name == 'live-mint'].sample(5).url.values

array(['https://www.livemint.com/market/live-blog/tata-chemicals-share-price-live-blog-for-16-nov-2023-11700102665178.html',
       'https://www.livemint.com/companies/elon-musk-s-neuralink-gets-fda-approval-for-study-of-brain-implants-in-humans-11685119212072.html',
       'https://www.livemint.com/companies/start-ups/kotak-investment-advisors-to-close-its-startup-fund-cites-falling-valuations-as-a-reason-11681394105602.html',
       'https://www.livemint.com/market/live-blog/indian-railway-catering-tourism-corporation-share-price-live-blog-for-25-oct-2023-11698202356607.html',
       'https://www.livemint.com/industry/media/adipurush-box-office-collection-movie-continues-to-mint-money-despite-controversy-11687157962872.html'],
      dtype=object)

In [102]:
articles_df[articles_df.source_name == 'navi'].sample(5).url.values

array(['https://navi.com/blog/hybrid-mutual-funds/',
       'https://navi.com/blog/tpa-third-party-administrator/',
       'https://navi.com/blog/telangana-land-records/',
       'https://navi.com/blog/yield-curve-inversion/',
       'https://navi.com/blog/equity-portfoliomanagement/'], dtype=object)

In [103]:
articles_df[articles_df.source_name == 'wint-wealth'].sample(5).url.values

array(['https://www.wintwealth.com/blog/munger-commercial-real-estate-investment',
       'https://www.wintwealth.com/blog/commercial-real-estate-investment-in-muzaffarpur',
       'https://www.wintwealth.com/blog/face-value-of-shares-meaning-working-importance',
       'https://www.wintwealth.com/blog/know-the-features-and-benefits-of-a-trading-account',
       'https://www.wintwealth.com/blog/get-sukanya-samriddhi-account-passbook-online'],
      dtype=object)

In [104]:
articles_df[articles_df.source_name == 'zerodha-varsity'].sample(5).url.values

array(['https://zerodha.com/varsity/the-history-of-indian-venture-capital/',
       'https://zerodha.com/varsity/chapter/dont-deliberate-just-do-it/',
       'https://zerodha.com/varsity/chapter/getting-ready-to-trade/',
       'https://zerodha.com/varsity/chapter/disappointment-and-regret-the-other-trading-emotions/',
       'https://zerodha.com/varsity/chapter/discipline-takes-practice/'],
      dtype=object)

In [21]:
from src.articles.ArticleService import ArticleService

In [22]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [23]:
import re

In [24]:
subcat_sources = ['live-mint','economic-times', 'cnbc-tv-18']
cat_sources = ['navi', 'zerodha-varsity', 'wint-wealth', 'investopedia', 'groww', 'business-world', 'bq-prime']

def extract_category_subcategory(row):
    # 20 and 25 identified based on iterative analysis
    source = row['source_name']
    x = row['url']
    if source in cat_sources:
        cat_url = re.split('.com/|.in/', x)[1]
        cat = cat_url.split('/')[0]
        if len(cat) < 20:
            return cat, ''
        else:
            return '',''
    elif source in subcat_sources:
        cat_url = re.split('.com/|.in/', x)[1]
        cat = cat_url.split('/')[0]
        sub_cat = cat_url.split('/')[1]
        if len(sub_cat) < 25:
            return cat, sub_cat
        else:
            return cat, ''
    else:
        return '', ''

In [25]:
articles_df[['cat', 'subcat']] = articles_df.apply(extract_category_subcategory, axis = 1, result_type='expand')

In [26]:
len('personal-finance-news')

21

In [27]:
articles_df.groupby('cat')['cat'].count().nsmallest()

cat
1099-g-form-4782667    1
after-8347322          1
ajay-banga-7494796     1
alphanomics-7974094    1
ameribor-6260927       1
Name: cat, dtype: int64

In [28]:
# removing those entries where there is only 1 cat count or 1 subcat count
invalid_cats = articles_df[articles_df.groupby('cat')['cat'].transform('count') == 1].cat.unique()

invalid_subcats = articles_df[articles_df.groupby('subcat')['subcat'].transform('count') == 1].subcat.unique()

articles_df.loc[articles_df.cat.isin(invalid_cats),'cat'] = ''
articles_df.loc[articles_df.subcat.isin(invalid_subcats),'subcat'] = ''

In [29]:
articles_df['fullcat'] = articles_df['source_name'] + '-' + articles_df['cat'] + '-' + articles_df['subcat']

In [30]:
len(articles_df.groupby('fullcat')['article_id'].count())

185

### Re-balancing categories for more even representation

In [31]:
fullcat_prob = articles_df.groupby('fullcat')['article_id'].count()/len(articles_df)

In [32]:
fullcat_prob.nlargest()

fullcat
cnbc--                                0.131152
investopedia--                        0.073991
economic-times-news-international     0.062416
investopedia-terms-                   0.054172
live-mint-market-stock-market-news    0.052459
Name: article_id, dtype: float64

In [167]:
reweighted_fullcat_prob = np.power(fullcat_prob, -0.33)/np.sum(np.power(fullcat_prob, -0.33))

In [168]:
reweighted_fullcat_prob['cnbc--']

0.0006711166081477496

In [169]:
reweighted_fullcat_prob

fullcat
bq-prime-business-                0.002253
bq-prime-economy-finance-         0.007290
bq-prime-ipos-                    0.004505
bq-prime-law-and-policy-          0.007548
bq-prime-markets-                 0.003314
                                    ...   
live-mint-technology-apps         0.012399
live-mint-technology-tech-news    0.010846
navi-blog-                        0.001335
wint-wealth-blog-                 0.001328
zerodha-varsity-varsity-          0.001677
Name: article_id, Length: 185, dtype: float64

In [170]:
reweighted_fullcat_prob_df = pd.DataFrame(reweighted_fullcat_prob).rename(columns = {'article_id': 'proportion'}).reset_index()

In [171]:
if 'proportion' in articles_df.columns:
    articles_df = articles_df.drop(columns = {'proportion'})
articles_df = pd.merge(articles_df, reweighted_fullcat_prob_df, how = 'left')

In [172]:
baselevel = np.sum(articles_df['proportion'])

In [173]:
num_articles = 1500

In [174]:
articles_df['scaled_proportion'] = articles_df['proportion'] * (num_articles/baselevel)

In [175]:
articles_df.scaled_proportion.sum()

1500.0

In [176]:
articles_df['prob'] = [np.random.rand() for i in range(len(articles_df))]

In [177]:
articles_df['is_selected'] = articles_df['prob'] < articles_df['scaled_proportion']

In [178]:
articles_df.is_selected.sum()

1503

In [179]:
articles_df[articles_df.is_selected == 1].groupby('source_name')['is_selected'].sum().nlargest(100)

source_name
economic-times      381
live-mint           379
investopedia        225
cnbc-tv-18          181
business-world      114
cnbc                103
navi                 25
wint-wealth          24
business-insider     22
zerodha-varsity      18
bq-prime             16
groww                15
Name: is_selected, dtype: int64

In [180]:
articles_df.groupby('source_name')['is_selected'].count().nlargest(100)

source_name
economic-times      23795
live-mint           23124
investopedia        18343
cnbc                13778
cnbc-tv-18          11032
business-world       6767
wint-wealth          1743
groww                1731
navi                 1714
business-insider     1646
zerodha-varsity       859
bq-prime              522
Name: is_selected, dtype: int64

In [181]:
articles_df[articles_df.is_selected == 1].groupby('fullcat')['is_selected'].sum().nlargest(100)

fullcat
cnbc--                                103
investopedia--                         70
economic-times-news-international      64
investopedia-terms-                    55
live-mint-market-stock-market-news     54
                                     ... 
live-mint-market-mark-to-market         3
live-mint-money-                        3
live-mint-news-india                    3
bq-prime-markets-                       2
business-world-special-features-        2
Name: is_selected, Length: 100, dtype: int64

In [182]:
len(articles_df[articles_df.is_selected == 1].groupby('fullcat')['is_selected'].sum())

133

In [183]:
articles_df[(articles_df.is_selected == 1) & (articles_df.cat == 'news')].groupby('subcat')['is_selected'].sum().nlargest(50)

subcat
international          64
india                  55
politics-and-nation    39
defence                18
sports                 18
new-updates            17
economy                16
elections              16
environment             6
science                 5
                        4
company                 4
how-to                  3
world                   1
Name: is_selected, dtype: int64

In [184]:
articles_df[articles_df.is_selected == 1].groupby('cat')['is_selected'].sum().nlargest(50)

cat
news                 266
market               204
                     195
industry             152
companies             70
business              65
blog                  64
terms                 55
economy               45
articles              40
markets               36
ask                   31
money                 27
opinion               22
banking-finance       18
varsity               18
corporate             17
world                 17
photos                16
personal-finance      14
top-stories           14
auto-news             13
cryptocurrency         9
finance                9
financial-edge         8
small-biz              8
stock-market           8
Opinion                6
velocity               6
property               5
agribusiness           4
wealth                 4
infographics           3
investing              3
ipos                   3
tech                   3
Industry               2
Money                  2
budget                 2
insights             

In [194]:
shortlisted_articles = articles_df[articles_df.is_selected == True]
shortlisted_articles = shortlisted_articles.sample(frac=1)

In [195]:
oversampled_blog_articles = articles_df[articles_df.source_name.isin(['groww', 'wint-wealth', 'zerodha-varsity', 'business-insider'])].groupby('source_name').sample(20)

In [196]:
shortlisted_articles = pd.concat([shortlisted_articles, oversampled_blog_articles])

In [190]:
shortlisted_articles

,article_id,url,source_id,authors,image_url,is_premium_article,last_updated_time,published_time,short_description,tags,...,prob,id,decay,source_name,cat,subcat,fullcat,scaled_proportion,is_selected,proportion
30152,65316a351e5cc42b1b13db99,https://www.livemint.com/industry/advertising/...,650046bd005149c49201269f,[Varuni Khosla],https://images.livemint.com/img/2023/02/14/160...,False,2023-02-14T15:40:52+05:30,2023-02-14T15:40:52+05:30,There will be a 15.5% increase in ad spends in...,"[digital advertising, ad spends, Wavemaker, Es...",...,0.026106,2,0.9,live-mint,industry,advertising,live-mint-industry-advertising,0.039024,True,0.003584
97625,6555cc054b13023f934a4124,https://www.cnbctv18.com/business/companies/ai...,6555992096fb32eef21a562d,[Anand Singha],https://images.cnbctv18.com/wp-content/uploads...,False,2023-10-18T15:03:43+00:00,2023-10-17T17:33:35+00:00,While Tony Fernandes' LinkedIn post initially ...,"[AirAsia CEO, Tony Fernandes, Shirtless meetin...",...,0.002965,14,0.9,cnbc-tv-18,business,companies,cnbc-tv-18-business-companies,0.013210,True,0.001213
103939,65597a4e4b13023f934af118,https://www.cnbc.com/2023/11/18/niro-sivanatha...,6555995596fb32eef21a562e,[Megan Sauer],https://image.cnbcfm.com/api/v1/image/10733602...,False,2023-11-18T14:00:01+00:00,2023-11-18T14:00:01+00:00,"There's an ""easy fix"" to becoming more persuas...","[Psychology,Leadership]",...,0.000754,15,0.9,cnbc,,,cnbc--,0.007308,True,0.000671
70841,651e2632a662d76276b8a69e,https://economictimes.indiatimes.com/news/indi...,6512cdcad01a9c8e86263e05,[ANI],https://economictimes.indiatimes.com/thumb/msi...,False,2023-05-01T11:59:00+05:30,2023-04-30T08:16:00+05:30,"Earthquake of Magnitude:4.1, Occurred on 30-0...","[earthquake, jammu, kashmir, National Center f...",...,0.001980,1,0.9,economic-times,news,india,economic-times-news-india,0.010293,True,0.000945
103829,655828f04b13023f934af086,https://www.bqprime.com/business/banks-to-need...,65559d4296fb32eef21a5630,[PTI],https://gumlet.assettype.com/bloombergquint%2F...,False,2023-11-17T17:43:06+00:00,2023-11-17T17:43:09+00:00,"The banking system will need Rs 84,000 crore o...","[PTI,bqfree]",...,0.006518,17,0.9,bq-prime,business,,bq-prime-business-,0.024534,True,0.002253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,652ebbd41e5cc42b1b139b06,https://zerodha.com/varsity/chapter/socially-r...,65291b1e9a2fbc229e5f29c9,[],https://zerodha.com/varsity/wp-content/uploads...,False,2020-12-21T06:09:30+0000,2020-12-21T06:09:30+0000,The investor is overweighted in power. He or ...,None,...,0.226553,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.018260,False,0.001677
581,652ebbe01e5cc42b1b139bb4,https://zerodha.com/varsity/chapter/crude-oil-...,65291b1e9a2fbc229e5f29c9,[],http://zerodha.com/varsity/wp-content/uploads/...,False,2022-03-30T01:45:09+0000,2022-03-30T01:45:09+0000,11.1 – Mapping companies I’m hoping that the p...,None,...,0.178623,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.018260,False,0.001677
149,652ebba11e5cc42b1b139958,https://zerodha.com/varsity/chapter/maintainin...,65291b1e9a2fbc229e5f29c9,[],https://zerodha.com/varsity/wp-content/uploads...,False,2020-03-19T07:22:25+0000,2020-03-19T07:22:25+0000,Successful traders have a carefree attitude wh...,None,...,0.578346,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.018260,False,0.001677
398,652ebb9d1e5cc42b1b139941,https://zerodha.com/varsity/chapter/the-fly-an...,65291b1e9a2fbc229e5f29c9,[],https://zerodha.com/varsity/wp-content/uploads...,False,2020-01-29T05:28:03+0000,2020-01-29T05:28:03+0000,As traders it’s vital that you solve trading p...,None,...,0.142492,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.018260,False,0.001677


In [191]:
shortlisted_articles.groupby('source_name')['url'].count()

source_name
bq-prime             16
business-insider     42
business-world      114
cnbc                103
cnbc-tv-18          181
economic-times      381
groww                35
investopedia        225
live-mint           379
navi                 25
wint-wealth          44
zerodha-varsity      38
Name: url, dtype: int64

In [200]:
def __generate_full_content(title: str, cleaned_text: str) -> str:
    return title + ': ' + cleaned_text

def __clean_article_content(article_content: str):
    # removing new line symbol
    article_content = article_content.replace('\n', '')
    article_content = article_content.replace('\t', '')
    # removing characters which are not part of alphabets, numbers, currencies, percent, dot, dash, brackets, space, slash
    article_content = re.sub(r"[^a-zA-Z0-9$€₹%\-:/()\[\]{} .]", "", article_content)
    return article_content

def get_article_text(article_id):
    article_data = ArticleService.get_article_text_from_s3(article_id)
    title = article_data['meta_data']['title']
    cleaned_text = article_data['cleaned_text']
    full_content = __generate_full_content(title, cleaned_text)
    cleaned_text = __clean_article_content(full_content)
    return cleaned_text

In [202]:
shortlisted_articles['full_content'] = shortlisted_articles['article_id'].progress_apply(lambda x: get_article_text(x))

  0%|          | 0/1583 [00:00<?, ?it/s]

In [203]:
shortlisted_articles['num_words'] = shortlisted_articles['full_content'].apply(lambda x: len(x.split(' ')))

In [283]:
pd.options.display.max_rows = 200

In [204]:
shortlisted_articles = shortlisted_articles.sample(frac=1)

In [205]:
len(shortlisted_articles)

1583

In [206]:
shortlisted_articles.to_csv('shortlisted_articles-30-Nov-23.csv')

In [207]:
test_set = shortlisted_articles[:250]

In [208]:
train_set = shortlisted_articles[250:]

### Creating the prompt

In [242]:
old_system_prompt = '''
You are the editor for a leading Indian financial news website. Your role is to evaluate incoming articles based on critical attributes to maintain the website's standard. Use the structure <attribute name> (<accepted value range>): <evaluation criteria> to assess articles:

1. evaluation_of_is_financial_news_or_not (short text of less than 20 words): Determine whether the article pertains to finance, excluding political, sports, entertainment, and other non-financial topics.
2. assessment_is_financial_news (1 or 0): 1 if it is a financial article, 0 if not
3. evaluation_of_relevant_for_india (short text of less than 20 words): Judge if the news article is relevant for indian readers. for example news articles about 401k is finance related but not for india. however changes to fed interest rates/nasdaq will be relevant for indian investors
4. assessment_relevant_for_india (1 or 0 ): 1 if it is relevant for indian readers, 0 if not
5. evaluation_of_is_article_safe (text between 10 words to 50 words): Judge if the article has any harmful, hateful, biased content. Check for blatent sponsored articles, opinions/recommendations contrary to well established financial practices. You can expect most of the articles to be completely safe since they will be merely facts or education pieces.
6. assessment_is_article_safe (1 or 0): 1 if the article is safe, 0 if not
7. evaluation_of_article_validity_duration (short text of less than 20 words): Judge for long will that article be relevant for the readers. For example, articles related to stock price fluctuations would be valid for 1 day. Earning call results probably for a few days. Important government policy changes or major economic events for a few weeks. Educational articles or opinion pieces on general practice will be timeless
8. assessment_article_validity_duration (one of 1, 3, 7, 14, 30, 60, -1): give score based on evaluation_of_article_validity_duration. -1 if the article is timeless. 1 if the article is relevant only for that day. 3 if for a couple of days and similarity 7, 14, 30, 60
9. evaluation_article_has_advanced_concepts (text between 10 to 50 words): Judge what level of technical knowledge is needed to read and comprehend the article. Readers on the platform understand stock movements, and usual business metrics. However, opinion or educational pieces need more technical financial knowledge to understand
10. assessment_article_has_advanced_concepts (1 or 0): 1 if the article needs advanced financial knowledge to understand and 0 if not
11. evaluation_of_likely_popularity (short text of less than 20 words): Judge how popular the article is likely going to be based how significant the news is, on how many people are going to be impacted by the news, quality of the article. minor stock fluctuations of various are likely less popular, and large fluctuations are more popular. minor government policy changes are likely to have medium popularity while important changes are going to be very popular. popularity of educational articles, opinion and analysis pieces is going to depend on the popularity of entities talked about and the quality of the content
12. assessment_likely_popularity (one of 1, 2, 3, 4, 5): give score based on evaluation_of_likely_popularity. 1 if the article is not likely to be popular, 5 if the article is likely to be very popular. be ruthless in enforcing a power law distribution on popularity - wherein most articles are likely to be 1, followed by 2, 3, 4, 5 in reducing popularity. 50% of the accessements should be 1, 20% should be 2, 15% should be 3, 10% should be 4, 5% should be 5
13. evaluation_of_article_type (short text of less than 20 words): Judge if the article majorly talks about facts, or is an opinion piece or is an education piece
14. assessment_article_type (one of factual, opinion, educational): if evaluation of evaluation_of_article_type indicates that article mainly talks about facts. opinion if it has mainly analysis or opinion piece. education if it an instructional and educational article
15. summary (text between 30 to 150 words): Give a succint summary of the article while being brief but retaining the key aspects of the article. don't critique the article itself or mention the article in third person. It should be a shorter summarized article
16. headline_suggestion (short text of less than 20 words): Recommend a headline based on the content of the article

all of your evaluations should capture the thought process behind the judgement while being brief. It should be the rationale on which the corresponding assessment attributes are based on and the evaluation should justify the assessment.

your response should be in a json structure with the above keys use double quotes (") and not single quotes (') for keys and values so that json can be decoded. no preamble or postamble. the following is the json structure that i expected:

{
"evaluation_of_is_financial_news_or_not": "",
"assessment_is_financial_news": "",
"evaluation_of_relevant_for_india": "",
"assessment_relevant_for_india": "",
"evaluation_of_is_article_safe": "",
"assessment_is_article_safe": "",
"evaluation_of_article_validity_duration": "",
"assessment_article_validity_duration": "",
"evaluation_article_has_advanced_concepts": "",
"assessment_article_has_advanced_concepts": "",
"evaluation_of_likely_popularity": "",
"assessment_likely_popularity": "",
"evaluation_of_article_type": "",
"assessment_article_type": "",
"summary": "",
"headline_suggestion": ""
}

|actual article text|

'''

In [5]:
system_prompt = '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (1/0) : <1 or 0 based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. summary_list (text of 60 words) : <Generate increasingly concise, entity-dense summaries. Repeat the following steps 3 times. 
Step a. Identify 1-3 informative Entities (semicolon delimited) from the Article which are missing from the previously generated summary. 
Step b. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities. 

A Missing Entity is: 
Relevant to the main story: Specific, descriptive yet concise (5 words or fewer); 
Novel: not in the previous summary; 
Faithful: present in the Article; 

Guidelines: Don't refer to the article in third person, but summarize the contents. The first summary should be long (4-5 sentences, ~60 words) yet highly non-specific. Subsequently rewrite the previous summary to improve flow and make space for additional entities. Make space with compression and removal of uninformative phrases like 'the article discusses'. The summaries should become highly dense but easily understood without the Article. Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. generate a list of 3 dictionaries whose keys are 'missing_entities' and 'denser_summary'>
14. headline_suggestion (short text) : <Write a headline based on the content of the article>
15. relevant_questions (list of 5 strings) : <List of 5 questions for which this article would be an answer. Each string should be a question>
16. relevant_search_queries (list of 5 strings) : <List of 5 google type search queries for which this article would be an answer. Don't make the queries too specific or too generic like finance, business, india>
17. keywords (list of 5 strings): <List of 5 important keywords for the article>

your response should be in a json structure with all the 17 above keys without missing any key. use (") and not (') for keys and values so that json can be decoded. no preamble or postamble.

|article|
'''

In [244]:
import re

In [245]:
len(re.split('_ \s', system_prompt))

1

In [246]:
len(re.split('_ ', system_prompt))

1

In [247]:
len(system_prompt.replace('_', ' ').split(' '))

812

### Calling GPT-4

In [57]:
import openai

In [58]:
openai.api_key = ''

In [250]:
shortlisted_articles.iloc[0].article_id

'65316faa1e5cc42b1b141024'

In [252]:
res1 = openai.ChatCompletion.create(model = "gpt-4", messages = [{'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': shortlisted_articles.iloc[0].full_content}])

In [253]:
import json

In [258]:
parsed_json = json.loads(res1['choices'][0]['message']['content'])

In [266]:
response_tuple = [('65316faa1e5cc42b1b141024', res1)]

In [268]:
pd.DataFrame(response_tuple, columns = ['article_id', 'response']).to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')

In [280]:
pd.DataFrame(k,index=['i',])

,article_id,system_prompt,user_prompt,model,content,prompt_tokens,completion_tokens
i,65316faa1e5cc42b1b141024,\nYou are the chief editor for a leading India...,Sebi halves listing time to three days Mint: ...,gpt-4-0613,"{'analysis_is_financial_or_business_news': ""Th...",1370,535


In [281]:
from tqdm.notebook import tqdm

In [284]:
# gpt35_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-16k")

In [283]:
# def find_token_limit(text, token_limit, l):
#     left = 0
#     right = len(text)

#     while left < right:
#         mid = left + (right - left) // 2
#         left_tokens = len(gpt35_encoding.encode(text[:mid]))
#         right_tokens = len(gpt35_encoding.encode(text[mid:]))

#         if left_tokens + right_tokens <= token_limit:
#             right = mid
#         else:
#             left = mid + 1

#     return left

In [8]:
def calculate_tokens(text, encoder):  # Placeholder for your actual token calculation function
    # Your implementation will go here.
    return len(encoder.encode(text))  # Example: counting characters as tokens

def find_token_limit(text,encoder, token_limit):
    # First, check if the whole text is under the token limit
    if calculate_tokens(text, encoder) <= token_limit:
        return text  # The entire text is within the limit

    def is_under_limit(index):
        # Use the provided function to calculate tokens for the substring
        return calculate_tokens(text[:index], encoder) <= token_limit

    left, right = 0, len(text)
    valid_limit = 0  # This will hold the index of the last valid token position

    # Binary search to find the token limit
    while left <= right:
        mid = (left + right) // 2  # Find the midpoint
        if is_under_limit(mid):
            # If the midpoint is under the limit, store it as a valid limit
            valid_limit = mid
            left = mid + 1  # Move the left boundary to the right
        else:
            right = mid - 1  # Move the right boundary to the left

    # Find the last space before the valid_limit to ensure we're at a word boundary
    space_index = text.rfind(' ', 0, valid_limit)
    if space_index == -1:
        # If there's no space, we've hit the start of the text
        return text[:valid_limit]  # Return up to the valid limit even if mid-word

    # Return the text up to the last word within the token limit
    return text[:space_index]

# Example usage:
text = "Your text goes here, and it could be very long."
token_limit = 100  # Replace with your actual token limit

In [461]:
find_token_limit(text, gpt35_encoding, token_limit=5)

'Your text goes'

In [437]:
len(tokens_integer)

3348

In [425]:
!pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/fb/2a/3d02ef030f387c373acbeca6d5a2307405a1da735285ec12a9ed0b6302ea/tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.4/924.4 kB 5.8 MB/s eta 0:00:0000:0100:01


In [9]:
import tiktoken

In [10]:
shorted_article_assessments = []

In [287]:
len(shorted_article_assessments)

0

In [288]:
len(shortlisted_articles)

826

In [11]:
model = 'gpt-4'

In [502]:
archived_system_prompt = '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (1/0) : <1 or 0 based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. summary_list (text of 60 words) : <Generate increasingly concise, entity-dense summaries. Repeat the following steps 3 times. 
Step a. Identify 1-3 informative Entities (semicolon delimited) from the Article which are missing from the previously generated summary. 
Step b. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities. 

A Missing Entity is: 
Relevant to the main story: Specific, descriptive yet concise (5 words or fewer); 
Novel: not in the previous summary; 
Faithful: present in the Article; 

Guidelines: Don't refer to the article in third person, but summarize the contents. The first summary should be long (4-5 sentences, ~60 words) yet highly non-specific. Subsequently rewrite the previous summary to improve flow and make space for additional entities. Make space with compression and removal of uninformative phrases like 'the article discusses'. The summaries should become highly dense but easily understood without the Article. Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. generate a list of 3 dictionaries whose keys are 'missing_entities' and 'denser_summary'. and put the entire list in double quotes (") for safe overall json decoding>
14. headline_suggestion (short text) : <Write a headline based on the content of the article>
15. relevant_questions (text of 5 questions) : <comma seperated 5 questions for which this article would be an answer. Each string should be a question>
16. relevant_search_queries (text of 5 queries) : <comma seperated 5 google type search queries for which this article would be an answer. Don't make the queries too specific or too generic like finance, business, india>
17. keywords (text of 5 keywords): <comma seperated 5 important keywords for the article>

your response should be a json structure with all the 17 above keys without missing any key. use (") and not (') for keys and values so that json can be decoded. no preamble or postamble. respond in the exact following structure:

{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"evaluation_of_article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"assessment_article_type": "",
"summary_list": "",
"headline_suggestion": "",
"relevant_questions": "",
"relevant_search_queries": "",
"keywords": ""}

|article|
'''

In [12]:
system_prompt =  '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (1/0) : <1 or 0 based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. summary_list (text of 60 words) : <Generate increasingly concise, entity-dense summaries. Repeat the following steps 3 times. 
Step a. Identify 1-3 informative Entities (semicolon delimited) from the Article which are missing from the previously generated summary. 
Step b. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities. 

A Missing Entity is: 
Relevant to the main story: Specific, descriptive yet concise (5 words or fewer); 
Novel: not in the previous summary; 
Faithful: present in the Article; 

Guidelines: Don't refer to the article in third person, but summarize the contents. The first summary should be long (4-5 sentences, ~60 words) yet highly non-specific. Subsequently rewrite the previous summary to improve flow and make space for additional entities. Make space with compression and removal of uninformative phrases like 'the article discusses'. The summaries should become highly dense but easily understood without the Article. Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. generate a list of 3 dictionaries whose keys are 'missing_entities' and 'denser_summary'. and put the entire list in double quotes (") for safe overall json decoding>
14. headline_suggestion (short text) : <Write a headline based on the content of the article>

your response should be a list of 2 json structures with all the 14 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:
[
{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"evaluation_of_article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"assessment_article_type": "",
"headline_suggestion": ""},
{"summary_list": ""}
]

|article|
'''

In [330]:
'''

{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"evaluation_of_article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"assessment_article_type": "",
"summary_list": "",
"headline_suggestion": "",
"relevant_questions": "",
"relevant_search_queries": "",
"keywords": ""
}
'''

'\n\n{\n"analysis_is_financial_or_business_news": "",\n"is_financial_or_business_news": "",\n"analysis_of_relevant_for_india": "",\n"relevant_for_india": "",\n"analysis_of_article_validity_duration": "",\n"article_validity_duration": "",\n"evaluation_of_article_validity_duration": "",\n"analysis_of_popularity": "",\n"popularity": "",\n"analysis_of_article_type": "",\n"article_type": "",\n"analysis_of_article_sentiment": "",\n"article_sentiment": "",\n"assessment_article_type": "",\n"summary_list": "",\n"headline_suggestion": "",\n"relevant_questions": "",\n"relevant_search_queries": "",\n"keywords": ""\n}\n'

In [332]:
system_prompt.replace('\n',' ')

' You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.  1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren\'t> 2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute> 3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won\'t be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant> 4. relevant_for_india (1/0) : <1 or 0 based on previous attribute> 5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: 

In [410]:
json.loads(res_gpt['choices'][0]['message']['content'].replace('\\\'',"'"))

{'analysis_is_financial_or_business_news': 'The article discusses a geo-political issue.',
 'is_financial_or_business_news': 0,
 'analysis_of_relevant_for_india': "Article covers a Ukraine-Russia conflict situation which isn't directly related to India.",
 'relevant_for_india': 0,
 'analysis_of_article_validity_duration': 'The situation in Ukraine is frequently changing, the article has short validity.',
 'article_validity_duration': 1,
 'analysis_of_popularity': 'The situation in Ukraine is a global issue, but the article covers a very specific strategic point which might limit the audience.',
 'popularity': 'moderately_popular',
 'analysis_of_article_type': 'This article purely details events in Ukraine, making it a factual piece.',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The situation described in the article is neutral in terms of market sentiment.',
 'article_sentiment': 'NA',
 'summary_list': '[{"missing_entities": "Russi\'s forces, supplying route, Bakhmut", 

In [378]:
res_gpt['choices'][0]['message']['content'][2150:2200]

'uers.\\" }]", "headline_suggestion": "Navigating th'

In [13]:
def parse_gpt_api_response(article_id, system_prompt, user_prompt, api_response):
    # Load the response into a dictionary if it's a string
    if isinstance(api_response, str):
        api_response = json.loads(api_response)
    
    # Extract the required information from the response
    id = api_response.get('id')
    model = api_response.get('model')
    prompt_tokens = api_response['usage'].get('prompt_tokens')
    completion_tokens = api_response['usage'].get('completion_tokens')

    # The content within the choices is a string of json, so it needs to be loaded
    replacers = {'\\\'':"'", '],':']",', ',[':',"['}
    try:
        content = json.loads(api_response['choices'][0]['message']['content'])
    except:
        print(f'trying to get fixed json for articleId {article_id}')
        improved_api_response = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': api_response['choices'][0]['message']['content']}])
        content = json.loads(improved_api_response['choices'][0]['message']['content'])
        # content = json.loads(api_response['choices'][0]['message']['content'].replace(replacers))
        # content = json.loads(api_response['choices'][0]['message']['content'].replace('],',']",').replace(',[',',"[').replace('\\\'',"'").replace('\\ "', '\\"'))
        # content = json.loads(api_response['choices'][0]['message']['content'])
    # Build the dictionary with the required information
    extracted_data = {
        'article_id': article_id,
        'system_prompt': system_prompt,
        'user_prompt': user_prompt,
        'model': model,
        'content': json.dumps(content),
        'prompt_tokens': prompt_tokens,
        'completion_tokens': completion_tokens
    }

    return extracted_data

In [447]:
r = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': 
                                                                                            '''fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble
                                                                                            '{ "analysis_is_financial_or_business_news": "The article focuses on the daily stock price activity of Aditya Birla Capital", "is_financial_or_business_news": "1", "analysis_of_relevant_for_india": "The article pertains to an Indian company\'s stock, hence relevant to Indian investors", "relevant_for_india": "1", "analysis_of_article_validity_duration": "Article pertains to daily stock price changes, hence it has a short relevance life", "article_validity_duration": "1", "analysis_of_popularity": "The article\'s content is pretty niche, centred around a specific company\'s stock\'s activity", "popularity": "niche", "analysis_of_article_type": "The article is factual, giving precise data regarding stock prices", "article_type": "fact", "analysis_of_article_sentiment": "The article shows no apparent positive or negative perspective towards the stock", "article_sentiment": "NA", "summary_list": "[{\\"missing_entities\\": \\"Aditya Birla Capital; stock price; 03 Nov 2023\\", \\"denser_summary\\": \\"Aditya Birla Capital\'s live stock price on 03 Nov 2023, opened at ₹171.75, reached a high of ₹173.3, and closed at ₹170.05 while its 52-week high is ₹199.4 and low is ₹115.85.\\"}, {\\"missing_entities\\": \\"BSE volume; Market capitalization\\", \\"denser_summary\\": \\"On 03 Nov 2023, Aditya Birla Capital\'s stock volume on BSE was 64553 shares; Market cap is ₹44866.44 crore. 52-week range is ₹115.85-₹199.4.\\"}, {\\"missing_entities\\": \\"LiveMint; AI-generated live blog\\", \\"denser_summary\\": \\"LiveMint\'s AI-generated blog: On 03 Nov 2023, Aditya Birla Capital\'s BSE stock volume was 64553; Market cap ₹44866.44 crore; 52-week range ₹115.85-₹199.4.\\"}]", "headline_suggestion": "Aditya Birla Capital\'s Stock Prices on 03 Nov 2023", "relevant_questions": "[\\"What was the closing price of Aditya Birla Capital\'s stock on 03 Nov 2023?\\", \\"What was the BSE volume for Aditya Birla Capital on 03 Nov 2023?\\", \\"What was Aditya Birla Capital\'s market capitalization on 03 Nov 2023?\\",\n\\"What are the 52-week high and low for Aditya Birla Capital?\\", \\"What were the opening and closing prices for Aditya Birla Capital on 03 Nov 2023?\\"]", "relevant_search_queries": "[\\"Aditya Birla Capital stock price on 03 Nov 2023\\", \\"BSE volume of Aditya Birla Capital on 03 Nov 2023\\", \\"52-week high & low for Aditya Birla Capital\\", \\"Market capitalization of Aditya Birla Capital on 03 Nov 2023\\", \\"Closing price of Aditya Birla Capital on 03 Nov 2023\\"]", "keywords": "[\\"Aditya Birla Capital\\", \\"BSE volume\\", \\"stock prices\\", \\"market capitalization\\", \\"52-week high & low\\"]" }'
                                                                                            '''}])

In [449]:
json.loads(r['choices'][0]['message']['content'])

{'analysis_is_financial_or_business_news': 'The article focuses on the daily stock price activity of Aditya Birla Capital',
 'is_financial_or_business_news': '1',
 'analysis_of_relevant_for_india': "The article pertains to an Indian company's stock, hence relevant to Indian investors",
 'relevant_for_india': '1',
 'analysis_of_article_validity_duration': 'Article pertains to daily stock price changes, hence it has a short relevance life',
 'article_validity_duration': '1',
 'analysis_of_popularity': "The article's content is pretty niche, centred around a specific company's stock's activity",
 'popularity': 'niche',
 'analysis_of_article_type': 'The article is factual, giving precise data regarding stock prices',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The article shows no apparent positive or negative perspective towards the stock',
 'article_sentiment': 'NA',
 'summary_list': [{'missing_entities': 'Aditya Birla Capital; stock price; 03 Nov 2023',
   'denser_summar

In [445]:
res_gpt['choices'][0]['message']['content']

'{ "analysis_is_financial_or_business_news": "The article focuses on the daily stock price activity of Aditya Birla Capital", "is_financial_or_business_news": "1", "analysis_of_relevant_for_india": "The article pertains to an Indian company\'s stock, hence relevant to Indian investors", "relevant_for_india": "1", "analysis_of_article_validity_duration": "Article pertains to daily stock price changes, hence it has a short relevance life", "article_validity_duration": "1", "analysis_of_popularity": "The article\'s content is pretty niche, centred around a specific company\'s stock\'s activity", "popularity": "niche", "analysis_of_article_type": "The article is factual, giving precise data regarding stock prices", "article_type": "fact", "analysis_of_article_sentiment": "The article shows no apparent positive or negative perspective towards the stock", "article_sentiment": "NA", "summary_list": "[{\\"missing_entities\\": \\"Aditya Birla Capital; stock price; 03 Nov 2023\\", \\"denser_summ

In [431]:
res_gpt['choices'][0]['message']['content'][2200:].replace('],',']",').replace(',[',',"[').replace('\\\'',"'")

'r QR codes on products?"]", "relevant_search_queries": "[\\"Delhi High Court PIL on QR codes\\", \\"Benefit of QR codes for visually impaired\\", \\"Kapila and Nirmal Hingorani Foundation PIL\\", \\"Aman Hingorani QR code PIL\\", \\"Centre\'s reaction to QR code PIL\\"]", "keywords": "[\\"PIL\\", \\"QR codes\\", \\"Delhi High Court\\", \\"visually impaired\\", \\"Centre\'s response\\"]" }'

In [460]:
import ast

In [15]:
shortlisted_articles[shortlisted_articles.article_id == '6555cc7d4b13023f934a987a']

,Unnamed: 0,article_id,url,source_id,authors,image_url,is_premium_article,last_updated_time,published_time,short_description,...,source_name,ends_with_html,cat,subcat,fullcat,proportion,scaled_proportion,is_selected,full_content,num_words
717,81790,6555cc7d4b13023f934a987a,https://www.cnbctv18.com/market/ten-year-treas...,6555992096fb32eef21a562d,['Bloomberg'],https://images.cnbctv18.com/wp-content/uploads...,False,2023-10-23T15:37:32+00:00,2023-10-23T15:35:32+00:00,The rise in the global bond benchmark above th...,...,cnbc-tv-18,False,market,NaN,cnbc-tv-18-market-,0.001088,0.005925,True,Ten-year treasury yield breaches 5% for first ...,517


In [17]:
from tqdm.notebook import tqdm

In [19]:
import json

In [20]:
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
total_tokens = 7000
available_tokens = total_tokens - num_system_tokens
total_rows = len(shortlisted_articles)
for i in tqdm(range(718, total_rows)):
    row = shortlisted_articles.iloc[i]
    truncated_content = find_token_limit(text=row.full_content, encoder=encoder, token_limit=available_tokens)
    res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': truncated_content}])
    parsed_res = parse_gpt_api_response(article_id=row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
    cur_df = pd.DataFrame(parsed_res, index = ['i',])
    cur_df.to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')
    shorted_article_assessments.append(parsed_res)

  0%|          | 0/108 [00:00<?, ?it/s]

trying to get fixed json for articleId 651e17b8a662d76276b88483
trying to get fixed json for articleId 65316ede1e5cc42b1b13e889
trying to get fixed json for articleId 6555cbe14b13023f934a283c
trying to get fixed json for articleId 65316e651e5cc42b1b13e4f2
trying to get fixed json for articleId 6555cbe74b13023f934a2c6c
trying to get fixed json for articleId 6531690b1e5cc42b1b13cb89
trying to get fixed json for articleId 6555c82b4b13023f9349150e


#### Analysis

In [24]:
all_article_responses = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [ ]:
all_article_responses.rename(columns = {'0'})

In [27]:
all_article_responses[1].nunique()

825

In [30]:
all_article_responses[6].sum() * 0.03 * 80/1000

4017.768

In [31]:
all_article_responses[7].sum() * 0.06 * 80/1000

2145.3984

In [32]:
invalid_response_article_ids = []

In [33]:
resp = []
for i in tqdm(range(len(all_article_responses))):
    row = all_article_responses.iloc[i]
    try:
        resp.append(json.loads(row[5]))
    except:
        invalid_response_article_ids.append(row[1])

  0%|          | 0/828 [00:00<?, ?it/s]

In [34]:
len(invalid_response_article_ids)

121

In [56]:
resp_df = pd.DataFrame(resp[:220])

In [61]:
resp_df['is_financial_or_business_news'] = resp_df['is_financial_or_business_news'].astype('int')
resp_df['relevant_for_india'] = resp_df['relevant_for_india'].astype('int')
resp_df['article_validity_duration'] = resp_df['article_validity_duration'].astype('int')

In [63]:
resp_df.is_financial_or_business_news.mean()

0.8863636363636364

In [64]:
resp_df.relevant_for_india.mean()

0.8090909090909091

In [69]:
resp_df.groupby('popularity')['popularity'].count()

popularity
breaking_news          33
moderately_popular    115
niche                  72
Name: popularity, dtype: int64

### Rerunning for invalid article responses

In [ ]:
invalid_response_article_ids

In [ ]:
shortlisted_articles

In [80]:
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
total_tokens = 7000
available_tokens = total_tokens - num_system_tokens
total_rows = len(shortlisted_articles)
for i in tqdm(invalid_response_article_ids[92:]):
    row = shortlisted_articles.loc[shortlisted_articles.article_id == i].iloc[0]
    truncated_content = find_token_limit(text=row.full_content, encoder=encoder, token_limit=available_tokens)
    res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': truncated_content}])
    parsed_res = parse_gpt_api_response(article_id=row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
    cur_df = pd.DataFrame(parsed_res, index = ['i',])
    cur_df.to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')
    shorted_article_assessments.append(parsed_res)

  0%|          | 0/29 [00:00<?, ?it/s]

In [500]:
df1 = pd.DataFrame(shorted_article_assessments)

In [503]:
df1['json_safe_content'] = df1['content'].apply(lambda x: json.dumps(x))

In [506]:
df1.to_csv('gpt-responses.csv')

In [510]:
df1['total_cost'] = df1['prompt_tokens'] * (0.03 * 80/1000) + df1['completion_tokens'] * (0.06 * 80/1000)

In [512]:
df1.total_cost.sum()

879.1272

In [495]:
pd.DataFrame(shorted_article_assessments).to_csv('gpt-responses.csv')

#### Standardizing responses

In [2]:
import pandas as pd
import numpy as np
import json

In [5]:
df = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [7]:
df = df.rename(columns = {1: 'article_id', 2: 'system_prompt', 3: 'article_content', 4: 'gpt-model', 5: 'gpt-response', 6: 'input_tokens', 7: 'output_tokens'})

In [157]:
article_wise_responses = {}

In [158]:
for i in range(len(df)):
    row = df.iloc[i]
    art_id = row['article_id']
    try:
        article_wise_responses[art_id] = {}
        article_wise_responses[art_id]['response'] = json.loads(row['gpt-response'])
        article_wise_responses[art_id]['content'] = row['article_content']
    except:
        pass

In [61]:
def correct_json_using_gpt(summary_json):
    print(f'trying to get fixed json for summary')
    improved_summary_json = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': summary_json}])
    content = json.loads(improved_summary_json['choices'][0]['message']['content'])
    return content

In [156]:
from tqdm.notebook import tqdm
import tqdm as td

from copy import deepcopy

In [185]:
response['summary_list']

[{'missing_entities': 'Nirmala Sitharaman; US India Business Council; Digitalization impact',
  'denser_summary': 'Union Finance Minister Nirmala Sitharaman urged American businesses to invest in India, asserting that it meets the requirements of a fair and transparent economy. The invitation came during a business roundtable organized by the US India Business Council. Sitharaman emphasized on the progress India has made in terms of digitalization and economic formalization.'},
 {'missing_entities': 'Taranjit Singh Sandhu; US and India trade; National Financial Information Registry',
  'denser_summary': "Indian Ambassador to the US, Taranjit Singh Sandhu reiterated India's commitment to engagement at the G20. He also noted that the US-India bilateral trade crossed $190 billion last year. The importance of a national financial information registry was emphasized to expand credit access."},
 {'missing_entities': 'Startup policies; Public Infrastructure; Fintech Ecosystem',
  'denser_summ

In [190]:
standardized_responses = {}
errenous_summaries = []
for art_id in td.tqdm(article_wise_responses.keys()):
    response = article_wise_responses[art_id]['response']
    expected_keys_1 = ['analysis_is_financial_or_business_news', 'is_financial_or_business_news', 'analysis_of_relevant_for_india', 'relevant_for_india',
                        'analysis_of_article_validity_duration', 'article_validity_duration', 'analysis_of_popularity', 'popularity', 'analysis_of_article_type', 'article_type', 
                        'analysis_of_article_sentiment', 'article_sentiment', 'headline_suggestion']
    expected_keys_2 = ['summary_list']
    if isinstance(response, list):
        res = {}
        for ele in response:
            res.update(ele)
        response = res
        # response = deepcopy(article_wise_responses[art_id]['response'][0])
        # try:
        #     response.update(article_wise_responses[art_id]['response'][1])
        # except:
        #     pass
        # assert sorted(set(response[0].keys()).intersection(expected_keys_1)) == sorted(expected_keys_1), f"{art_id}"
        # assert set(response[1].keys()) == expected_keys_2
        # try:
        #     summaries = json.loads(response[1]['summary_list'])
        # except:
        #     errenous_summaries.append(art_id)
        #     continue
        #     # print(f'articleId is {art_id}')
        #     # summaries = correct_json_using_gpt(summary_json = response[1]['summary_list'])
        # assert isinstance(summaries, list)
        # assert len(summaries) == 3
        # for summary in summaries:
        #     assert 'missing_entities' in summary and 'denser_summary' in summary
        # standardized_responses[art_id] = []
        # standardized_responses[art_id].append(response[1])
        # standardized_responses[art_id].append(summaries)
        # elif isinstance(response, dict):
    try:
        assert sorted(set(response.keys()).intersection(set(expected_keys_1).union(set(expected_keys_2)))) == sorted(set(expected_keys_1).union(set(expected_keys_2)))
    except:
        errenous_summaries.append(art_id)
        print(f'{art_id} error 0')
        continue
        # print(f'articleId is {art_id}')
        # summaries = correct_json_using_gpt(summary_json = response['summary_list'])
    try:
        if isinstance(response['summary_list'], str):
            summaries = json.loads(response['summary_list'])
        elif isinstance(response['summary_list'], list):
            summaries = response['summary_list']
    except:
        corrected_summary = correct_json_using_gpt(response['summary_list'])
        try:
            summaries = corrected_summary
        except:
            errenous_summaries.append(art_id)
            print(f'{art_id} error 1')
            continue
    assert isinstance(summaries, list)
    try:
        assert len(summaries) == 3
    except:
        print(f'{art_id} error 2')
        errenous_summaries.append(art_id)
        continue
    for summary in summaries:
        try:
            assert 'missing_entities' in summary and ('denser_summary' in summary or 'summary_text' in summary)
            if 'summary_text' in summary:
                summary['denser_summary'] = summary.pop('summary_text')
        except:
            print(f'{art_id} error 3')
            errenous_summaries.append(art_id)
            continue
    standardized_responses[art_id] = {}
    standardized_responses[art_id]['response'] = []
    attribute_response = {}
    for key in expected_keys_1:
        attribute_response[key] = response[key]
    standardized_responses[art_id]['response'].append(attribute_response)
    standardized_responses[art_id]['response'].append(summaries)
    standardized_responses[art_id]['content'] = article_wise_responses[art_id]['content']

  0%|          | 0/825 [00:00<?, ?it/s]

trying to get fixed json for summary


  0%|          | 3/825 [00:15<1:12:02,  5.26s/it]

trying to get fixed json for summary


  0%|          | 4/825 [00:35<2:16:53, 10.00s/it]

...

100%|█████████▉| 821/825 [1:07:31<00:27,  6.81s/it]

trying to get fixed json for summary


100%|█████████▉| 822/825 [1:07:55<00:27,  9.17s/it]

trying to get fixed json for summary


100%|██████████| 825/825 [1:08:09<00:00,  4.96s/it]


In [163]:
list(standardized_responses.keys())[-20:]

['6555ccc44b13023f934acd61',
 '651e0c73a662d76276b86a15',
 '651dd71da662d76276b7e3c2',
 '6555cb9f4b13023f9349fa71',
 '6555cba44b13023f9349fdd1',
 '6555cab84b13023f93495f7c',
 '6555c7ec4b13023f9348ee4a',
 '654cbbcadc4fa72a6c403e10',
 '6555c0084b13023f9348c97b',
 '6555ccd14b13023f934ad8ab',
 '651df9b1a662d76276b83c5d',
 '6555cbf64b13023f934a3718',
 '65316f1c1e5cc42b1b13efc1',
 '651e088ba662d76276b86119',
 '653168bd1e5cc42b1b13b78d',
 '655c5595eee55a44e0ac1e51',
 '6555c84a4b13023f934924ce',
 '651dca7ca662d76276b7c1c9',
 '651df878a662d76276b83979',
 '6555c80e4b13023f934901a6']

In [191]:
len(standardized_responses.keys())

821

In [192]:
len(errenous_summaries)

4

In [195]:
test_article_ids = list(df.iloc[:101].article_id.values)

In [196]:
len(set(test_article_ids).intersection(set(standardized_responses.keys())))

100

In [197]:
train_article_ids = [x for x in standardized_responses.keys() if x not in test_article_ids]

In [201]:
len(train_article_ids)

721

In [202]:
train_standardized_responses = {k: standardized_responses[k] for k in train_article_ids}

In [203]:
len(train_standardized_responses)

721

In [204]:
with open('train_set_full_27_nov.json', 'w') as f:
    json.dump(train_standardized_responses, f)

In [205]:
test_standardized_responses = {k: standardized_responses[k] for k in test_article_ids}

In [206]:
with open('test_set_full_27_nov.json', 'w') as f:
    json.dump(test_standardized_responses, f)